# HATLAS-SGP master catalogue

This notebook presents the merge of the various pristine catalogues to produce HELP mater catalogue on GAMA-09.

In [ ]:
from herschelhelp_internal import git_version
print("This notebook was run with herschelhelp_internal version: \n{}".format(git_version()))

In [ ]:
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))

import os
import time

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import Column, Table
import numpy as np
from pymoc import MOC

from herschelhelp_internal.masterlist import merge_catalogues, nb_merge_dist_plot, specz_merge
from herschelhelp_internal.utils import coords_to_hpidx, ebv, gen_help_id, inMoc

In [ ]:
TMP_DIR = os.environ.get('TMP_DIR', "./data_tmp")
OUT_DIR = os.environ.get('OUT_DIR', "./data/tiles")
SUFFIX = os.environ.get('SUFFIX', time.strftime("_%Y%m%d"))

try:
    os.makedirs(OUT_DIR)
except FileExistsError:
    pass

## I - Reading the prepared pristine catalogues

In [ ]:
#atlas = Table.read("{}/ATLAS.fits".format(TMP_DIR))["atlas_id", "atlas_ra", "atlas_dec",
#                                                   "atlas_stellarity",
#                                                   "atlas_flag_cleaned",
#                                                   "atlas_flag_gaia"]
#kids = Table.read("{}/KIDS.fits".format(TMP_DIR))["kids_id", "kids_ra", "kids_dec", 
#                                                  "kids_stellarity", 
#                                                  "kids_flag_cleaned",
#                                                  "kids_flag_gaia"]
omegacam = Table.read("{}/omegacam_merged_hatlas-sgp.fits".format(TMP_DIR))[
                                                   "omegacam_intid", "omegacam_ra", "omegacam_dec",
                                                   "atlas_id",
                                                   "atlas_stellarity",
                                                   "atlas_flag_cleaned",
                                                   "atlas_flag_gaia",
                                                   "kids_id", 
                                                   "kids_stellarity", 
                                                   "kids_flag_cleaned",
                                                   "kids_flag_gaia"]
ps1 = Table.read("{}/PS1.fits".format(TMP_DIR))["ps1_id","ps1_ra","ps1_dec","ps1_flag_gaia"]
viking = Table.read("{}/VISTA-VIKING.fits".format(TMP_DIR))["viking_id","viking_ra","viking_dec",
                                                           "viking_stellarity", 
                                                            "viking_flag_cleaned",
                                                            "viking_flag_gaia"]
des =  Table.read("{}/DES.fits".format(TMP_DIR))["des_id","des_ra","des_dec",
                                                 "des_flag_gaia",
                                                 "des_stellarity",
                                                 "des_flag_cleaned"]

## II - Merging tables

We first merge the optical catalogues and then add the infrared ones: CFHTLenS, CFHTLS, DECaLS, HSC, KIDS, PanSTARRS, UKIDSS-LAS, VISTA-VHS, and VISTA-VIKING.

At every step, we look at the distribution of the distances to the nearest source in the merged catalogue to determine the best crossmatching radius.

### Omegacam (ATLAS and KIDS)

In [ ]:
master_catalogue = omegacam
master_catalogue['omegacam_ra'].name = 'ra'
master_catalogue['omegacam_dec'].name = 'dec'

### Add PanSTARRS

In [ ]:
nb_merge_dist_plot(
    SkyCoord(master_catalogue['ra'], master_catalogue['dec']),
    SkyCoord(ps1['ps1_ra'], ps1['ps1_dec'])
)

In [ ]:
# Given the graph above, we use 0.8 arc-second radius
master_catalogue = merge_catalogues(master_catalogue, ps1, "ps1_ra", "ps1_dec", radius=0.8*u.arcsec)

### Add VIKING

In [ ]:
nb_merge_dist_plot(
    SkyCoord(master_catalogue['ra'], master_catalogue['dec']),
    SkyCoord(viking['viking_ra'], viking['viking_dec'])
)

In [ ]:
# Given the graph above, we use 1 arc-second radius
master_catalogue = merge_catalogues(master_catalogue, viking, "viking_ra", "viking_dec", radius=1.*u.arcsec)

### Add DES

In [ ]:
nb_merge_dist_plot(
    SkyCoord(master_catalogue['ra'], master_catalogue['dec']),
    SkyCoord(des['des_ra'], des['des_dec'])
)

In [ ]:
# Given the graph above, we use 0.8 arc-second radius
master_catalogue = merge_catalogues(master_catalogue, des, "des_ra", "des_dec", radius=0.8*u.arcsec)

### Cleaning

When we merge the catalogues, astropy masks the non-existent values (e.g. when a row comes only from a catalogue and has no counterparts in the other, the columns from the latest are masked for that row). We indicate to use NaN for masked values for floats columns, False for flag columns and -1 for ID columns.

In [ ]:
for col in master_catalogue.colnames:
    if "m_" in col or "merr_" in col or "f_" in col or "ferr_" in col or "stellarity" in col:
        master_catalogue[col].fill_value = np.nan
    elif "flag" in col:
        master_catalogue[col].fill_value = 0
    elif "id" in col:
        master_catalogue[col].fill_value = -1
        
master_catalogue = master_catalogue.filled()

In [ ]:
master_catalogue[:10].show_in_notebook()

## III - Merging flags and stellarity

Each pristine catalogue contains a flag indicating if the source was associated to a another nearby source that was removed during the cleaning process.  We merge these flags in a single one.

In [ ]:
flag_cleaned_columns = [column for column in master_catalogue.colnames
                        if 'flag_cleaned' in column]

flag_column = np.zeros(len(master_catalogue), dtype=bool)
for column in flag_cleaned_columns:
    flag_column |= master_catalogue[column]
    
master_catalogue.add_column(Column(data=flag_column, name="flag_cleaned"))
master_catalogue.remove_columns(flag_cleaned_columns)

Each pristine catalogue contains a flag indicating the probability of a source being a Gaia object (0: not a Gaia object, 1: possibly, 2: probably, 3: definitely).  We merge these flags taking the highest value.

In [ ]:
flag_gaia_columns = [column for column in master_catalogue.colnames
                     if 'flag_gaia' in column]

master_catalogue.add_column(Column(
    data=np.max([master_catalogue[column] for column in flag_gaia_columns], axis=0),
    name="flag_gaia"
))
master_catalogue.remove_columns(flag_gaia_columns)

Each prisitine catalogue may contain one or several stellarity columns indicating the probability (0 to 1) of each source being a star.  We merge these columns taking the highest value.

In [ ]:
stellarity_columns = [column for column in master_catalogue.colnames
                      if 'stellarity' in column]

print(", ".join(stellarity_columns))

In [ ]:

# We create an masked array with all the stellarities and get the maximum value, as well as its
# origin.  Some sources may not have an associated stellarity.
stellarity_array = np.array([master_catalogue[column] for column in stellarity_columns])
stellarity_array = np.ma.masked_array(stellarity_array, np.isnan(stellarity_array))

max_stellarity = np.max(stellarity_array, axis=0)
max_stellarity.fill_value = np.nan

no_stellarity_mask = max_stellarity.mask

master_catalogue.add_column(Column(data=max_stellarity.filled(), name="stellarity"))

stellarity_origin = np.full(len(master_catalogue), "NO_INFORMATION", dtype="S20")
stellarity_origin[~no_stellarity_mask] = np.array(stellarity_columns)[np.argmax(stellarity_array, axis=0)[~no_stellarity_mask]]

master_catalogue.add_column(Column(data=stellarity_origin, name="stellarity_origin"))

master_catalogue.remove_columns(stellarity_columns)

## IV - Adding E(B-V) column

In [ ]:
master_catalogue.add_column(
    ebv(master_catalogue['ra'], master_catalogue['dec'])
)

## V - Adding HELP unique identifiers and field columns

In [ ]:
master_catalogue.add_column(Column(gen_help_id(master_catalogue['ra'], master_catalogue['dec']),
                                   name="help_id"))
master_catalogue.add_column(Column(np.full(len(master_catalogue), "HATLAS-SGP", dtype='<U18'),
                                   name="field"))

In [ ]:
# Check that the HELP Ids are unique
if len(master_catalogue) != len(np.unique(master_catalogue['help_id'])):
    print("The HELP IDs are not unique!!!")
else:
    print("OK!")

## VII.a Wavelength domain coverage

We add a binary `flag_optnir_obs` indicating that a source was observed in a given wavelength domain:

- 1 for observation in optical;
- 2 for observation in near-infrared;
- 4 for observation in mid-infrared (IRAC).

It's an integer binary flag, so a source observed both in optical and near-infrared by not in mid-infrared would have this flag at 1 + 2 = 3.

*Note 1: The observation flag is based on the creation of multi-order coverage maps from the catalogues, this may not be accurate, especially on the edges of the coverage.*

*Note 2: Being on the observation coverage does not mean having fluxes in that wavelength domain. For sources observed in one domain but having no flux in it, one must take into consideration de different depths in the catalogue we are using.*

In [ ]:
atlas_moc = MOC(filename="../../dmu0/dmu0_ATLAS/data/ATLAS_HATLAS-SGP_MOC.fits")
kids_moc = MOC(filename="../../dmu0/dmu0_KIDS/data/KIDS-DR3_HATLAS-SGP_MOC.fits")
ps1_moc = MOC(filename="../../dmu0/dmu0_PanSTARRS1-3SS/data/PanSTARRS1-3SS_HATLAS-SGP_MOC.fits")
viking_moc = MOC(filename="../../dmu0/dmu0_VISTA-VIKING/data/VIKING_SGP_MOC.fits")
des_moc = MOC(filename="../../dmu0/dmu0_DES/data/DES-DR1_HATLAS-SGP_MOC.fits")

In [ ]:
was_observed_optical = inMoc(
    master_catalogue['ra'], master_catalogue['dec'],
    atlas_moc + kids_moc + ps1_moc + des_moc) 

was_observed_nir = inMoc(
    master_catalogue['ra'], master_catalogue['dec'],
    viking_moc
)

was_observed_mir = np.zeros(len(master_catalogue), dtype=bool)

#was_observed_mir = inMoc(
#    master_catalogue['ra'], master_catalogue['dec'],   
#)

In [ ]:
master_catalogue.add_column(
    Column(
        1 * was_observed_optical + 2 * was_observed_nir + 4 * was_observed_mir,
        name="flag_optnir_obs")
)

## VII.b Wavelength domain detection

We add a binary `flag_optnir_det` indicating that a source was detected in a given wavelength domain:

- 1 for detection in optical;
- 2 for detection in near-infrared;
- 4 for detection in mid-infrared (IRAC).

It's an integer binary flag, so a source detected both in optical and near-infrared by not in mid-infrared would have this flag at 1 + 2 = 3.

*Note 1: We use the total flux columns to know if the source has flux, in some catalogues, we may have aperture flux and no total flux.*

To get rid of artefacts (chip edges, star flares, etc.) we consider that a source is detected in one wavelength domain when it has a flux value in **at least two bands**. That means that good sources will be excluded from this flag when they are on the coverage of only one band.

This now takes place in the fonal stage when photometry is folded in


## VIII - Cross-identification table

We are producing a table associating to each HELP identifier, the identifiers of the sources in the pristine catalogue. This can be used to easily get additional information from them.


In [ ]:
id_names = []
for col in master_catalogue.colnames:
    if '_id' in col:
        id_names += [col]
    if '_intid' in col:
        id_names += [col]
        
print(id_names)

In [ ]:
master_catalogue[id_names].write(
    "{}/master_list_cross_ident_hatlas-sgp{}.fits".format(OUT_DIR, SUFFIX), overwrite=True)
id_names.remove('help_id')
#master_catalogue.remove_columns(id_names)

## IX - Adding HEALPix index

We are adding a column with a HEALPix index at order 13 associated with each source.

In [ ]:
master_catalogue.add_column(Column(
    data=coords_to_hpidx(master_catalogue['ra'], master_catalogue['dec'], order=13),
    name="hp_idx"
))

## X - Saving the catalogue

In [ ]:
columns = ["help_id", "field", "ra", "dec", "hp_idx"]

bands = [column[5:] for column in master_catalogue.colnames if 'f_ap' in column]
for band in bands:
    columns += ["f_ap_{}".format(band), "ferr_ap_{}".format(band),
                "m_ap_{}".format(band), "merr_ap_{}".format(band),
                "f_{}".format(band), "ferr_{}".format(band),
                "m_{}".format(band), "merr_{}".format(band),
                "flag_{}".format(band)]    
    
columns += ["stellarity", "flag_cleaned", "flag_merged", "flag_gaia", "flag_optnir_obs", "flag_optnir_det", "ebv"]

In [ ]:
# We check for columns in the master catalogue that we will not save to disk.
print("Missing columns: {}".format(set(master_catalogue.colnames) - set(columns)))

In [ ]:
#master_catalogue[columns].write("{}/master_catalogue_hatlas-sgp{}.fits".format(OUT_DIR, SUFFIX))

## XI - Folding in photometry
On HATLAS-SGP there is too much data to load all in to memory at once so we perform the cross matching without photometry columns. Only now do we fold in the photometry data by first cutting the catalogue up in to manageable sizes.

In [ ]:
split_length = 100000 #number of sources to include in every sub catalogue
num_files = int(np.ceil(len(master_catalogue)/split_length))
print(num_files)

In [ ]:
surveys = [
    #['atlas',    "ATLAS.fits" ,     Table.read("{}/ATLAS.fits".format(TMP_DIR)   ), "atlas_id"],    
    #['kids',    "KIDS.fits" ,   Table.read("{}/KIDS.fits".format(TMP_DIR) ), "kids_id"],     
    ['omegacam', "omegacam_merged_hatlas-sgp.fits" ,      Table.read("{}/omegacam_merged_hatlas-sgp.fits".format(TMP_DIR)    ), "omegacam_intid"],       
    ['ps1',    "PS1.fits" ,         Table.read("{}/PS1.fits".format(TMP_DIR)       ), "ps1_id"],
    ['viking',    "VIKING.fits" ,  Table.read("{}/VIKING.fits".format(TMP_DIR)), "viking_id"],
    ['des',    "DES.fits" ,  Table.read("{}/DES.fits".format(TMP_DIR)), "des_id"]
]

In [ ]:
#Sort catalogue by HELP id so that it is split up in RA strips
master_catalogue.sort('help_id')

In [ ]:
n=0
for sub_file in range(num_files):
    # the following used to have a -1 which was wrong as it left out objects
    sub_catalogue = master_catalogue[n*split_length:(n+1)*split_length] 
    #print(n)
    for survey in surveys:
        #print(survey[0])
        sub_catalogue = join(sub_catalogue, 
                               survey[2], #Table.read("{}/{}".format(TMP_DIR, survey[1])),
                               join_type='left',
                               metadata_conflicts='silent',
                               keys=survey[3]
                            )
    #print('Finished join')
    #sub_catalogue.remove_columns(id_names)
    
    
    for col in sub_catalogue.colnames:
        if (col.startswith("m_") or col.startswith("merr_") or col.startswith("f_") or col.startswith("ferr_")):
            sub_catalogue[col] = sub_catalogue[col].astype(float)
            sub_catalogue[col].fill_value = np.nan
        if "stellarity" in col:
            sub_catalogue[col].fill_value = np.nan
        elif "flag" in col:
            sub_catalogue[col].fill_value = 0
        elif "id" in col:
            sub_catalogue[col].fill_value = -1
        
    sub_catalogue = sub_catalogue.filled()
    
    
    #Adding detection flag
    
    # SpARCS is a catalogue of sources detected in r (with fluxes measured at 
    # this prior position in the other bands).  Thus, we are only using the r
    # CFHT band.
    # Check to use catalogue flags from HSC and PanSTARRS.
    nb_optical_flux = (
        #Omegacam 
        1 * ~np.isnan(master_catalogue['f_omegacam_ul']) +
        1 * ~np.isnan(master_catalogue['f_omegacam_u']) +
        1 * ~np.isnan(master_catalogue['f_omegacam_g']) +
        1 * ~np.isnan(master_catalogue['f_omegacam_r']) +
        1 * ~np.isnan(master_catalogue['f_omegacam_i']) +
        1 * ~np.isnan(master_catalogue['f_omegacam_z']) +
        # PanSTARRS
        1 * ~np.isnan(master_catalogue['f_gpc1_g']) +
        1 * ~np.isnan(master_catalogue['f_gpc1_r']) +
        1 * ~np.isnan(master_catalogue['f_gpc1_i']) +
        1 * ~np.isnan(master_catalogue['f_gpc1_z']) +
        1 * ~np.isnan(master_catalogue['f_gpc1_y']) +
        #DES
        1 * ~np.isnan(master_catalogue['f_decam_g']) +
        1 * ~np.isnan(master_catalogue['f_decam_r']) +    
        1 * ~np.isnan(master_catalogue['f_decam_i']) +
        1 * ~np.isnan(master_catalogue['f_decam_z']) +
        1 * ~np.isnan(master_catalogue['f_decam_y']) 
    )

    nb_nir_flux = (
        1 * ~np.isnan(master_catalogue['f_vista_z']) +
        1 * ~np.isnan(master_catalogue['f_vista_y']) +
        1 * ~np.isnan(master_catalogue['f_vista_j']) +
        1 * ~np.isnan(master_catalogue['f_vista_h']) +
        1 * ~np.isnan(master_catalogue['f_vista_k'])
    )

    nb_mir_flux = np.zeros(len(master_catalogue), dtype=bool)


    has_optical_flux = nb_optical_flux >= 2
    has_nir_flux = nb_nir_flux >= 2
    has_mir_flux = nb_mir_flux >= 2

    sub_catalogue.add_column(
    Column(
        1 * has_optical_flux + 2 * has_nir_flux + 4 * has_mir_flux,
        name="flag_optnir_det")
        )
    
    
    sub_catalogue.remove_columns(id_names)
    
    sub_catalogue.write("{}/sub_catalogue_sgp{}_{}.fits".format(OUT_DIR, SUFFIX, n), overwrite=True)
    n += 1

In [ ]:
sub_catalogue.colnames